# Layer Visualizations
This notebook contains code to parse a saved TensorFlow model file and generate a pandas dataframe that contains information about all the layers.

In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
import plotly.express as px
from pathlib import Path

# enable plotly in VS Studio Code
import plotly.io as pio
#pio.renderers.default = "notebook_connected"
pio.renderers.default = "plotly_mimetype+notebook"

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

from workbench.utils.utils import create_filepaths, parse_model_name, append_dict_to_csv
from workbench.config.config import initialize
from workbench.tflite_benchmarking import get_profiling_stats_cpu, get_profiling_dataframes_cpu

In [2]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:
model_name = "shufflenetv2tiny_0.1_96_c3_o3_f4l1024"
#model_name = "mobilenetv2_0.1_96_c3_o3_keras"
#model_name = "efficientNetB0_0.1_96_c3_o3_keras"

In [4]:
model_dict = {}

In [5]:
parse_model_name(model_name)

('shufflenetv2tiny', '0.1', '96', 'c3', 'o3', 'f4l1024')

In [6]:
models_dir = initialize()

In [7]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [8]:
tflite_benchmark_cpu_path = models_dir.joinpath(model_name, f"{model_name}_benchmark.txt")

In [9]:
tflite_model_stats_cpu = get_profiling_stats_cpu(tflite_benchmark_cpu_path)
tflite_model_stats_cpu

{'model_size_MB': 0.26636,
 'init_us': 164461,
 'first_inference_us': 12212,
 'warmup_avg_us': 426.317,
 'inference_avg_us': 550.678,
 'initialization_ms': 164.461,
 'modify_graph_with_delegate_ms_first': 8.256,
 'modify_graph_with_delegate_ms_avg': 8.256,
 'modify_graph_with_delegate_ms_%': 98.426,
 'modify_graph_with_delegate_mem_KB': 1096.0,
 'allocate_tensors_ms_first': 0.132,
 'allocate_tensors_ms_avg': 0.132,
 'allocate_tensors_ms_%': 1.574}

In [10]:
operator_run_order_df, operator_by_comp_time_df, summary_node_type_df = get_profiling_dataframes_cpu(tflite_benchmark_cpu_path, model_name)

In [11]:
operator_run_order_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,QUANTIZE,0.000,0.001,0.190,0.190,0.0,1,[StatefulPartitionedCall:0]:138
2,SOFTMAX,0.001,0.001,0.243,0.433,0.0,1,[StatefulPartitionedCall:01]:137
3,"Fully Connected (NC, QS8) GEMM",0.001,0.001,0.127,0.560,0.0,1,"Delegate/Fully Connected (NC, QS8) GEMM:2"
4,"Global Average Pooling (NWC, QS8)",0.001,0.001,0.325,0.885,0.0,1,"Delegate/Global Average Pooling (NWC, QS8):1"
5,RESHAPE,0.001,0.001,0.126,1.011,0.0,1,[shufflenetv2tiny/tf.reshape_31/Reshape]:133
6,RESHAPE,0.001,0.001,0.137,1.148,0.0,1,[shufflenetv2tiny/tf.reshape_30/Reshape]:131
7,RESHAPE,0.000,0.001,0.133,1.281,0.0,1,[shufflenetv2tiny/tf.reshape_29/Reshape]:125
8,RESHAPE,0.000,0.001,0.149,1.429,0.0,1,[shufflenetv2tiny/tf.reshape_28/Reshape]:123
9,RESHAPE,0.001,0.001,0.132,1.562,0.0,1,[shufflenetv2tiny/tf.reshape_27/Reshape]:117
10,RESHAPE,0.001,0.001,0.142,1.703,0.0,1,[shufflenetv2tiny/tf.reshape_26/Reshape]:115


In [12]:
operator_by_comp_time_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,QUANTIZE,0.110,0.125,30.023,30.023,0.0,1,[tfl.quantize]:0
2,"Convolution (NHWC, QC8) IGEMM",0.047,0.056,13.441,43.464,0.0,1,"Delegate/Convolution (NHWC, QC8) IGEMM:0"
3,"Convolution (NHWC, QC8) GEMM",0.004,0.012,8.694,52.158,0.0,3,"Delegate/Convolution (NHWC, QC8) GEMM:0"
4,"Max Pooling (NHWC, S8)",0.005,0.006,1.372,53.530,0.0,1,"Delegate/Max Pooling (NHWC, S8):1"
5,"Convolution (NHWC, QC8) DWConv",0.004,0.004,1.025,54.555,0.0,1,"Delegate/Convolution (NHWC, QC8) DWConv:5"
6,"Convolution (NHWC, QC8) GEMM",0.007,0.004,2.649,57.204,0.0,3,"Delegate/Convolution (NHWC, QC8) GEMM:2"
7,"Transpose (ND, X8)",0.005,0.003,12.252,69.456,0.0,16,"Delegate/Transpose (ND, X8):0"
8,"Convolution (NHWC, QC8) GEMM",0.002,0.003,0.642,70.098,0.0,1,"Delegate/Convolution (NHWC, QC8) GEMM:6"
9,"Convolution (NHWC, QC8) DWConv",0.004,0.002,6.864,76.962,0.0,13,"Delegate/Convolution (NHWC, QC8) DWConv:2"
10,"Convolution (NHWC, QC8) DWConv",0.003,0.002,1.420,78.382,0.0,3,"Delegate/Convolution (NHWC, QC8) DWConv:3"


In [13]:
summary_node_type_df

,Node type,count,avg ms,avg %,cdf %,mem KB,times called
1,QUANTIZE,2,0.124,32.208,32.208,0.0,2
2,"Convolution (NHWC, QC8) GEMM",6,0.094,24.416,56.623,0.0,36
3,"Convolution (NHWC, QC8) IGEMM",1,0.055,14.286,70.909,0.0,1
4,"Transpose (ND, X8)",1,0.050,12.987,83.896,0.0,16
5,"Convolution (NHWC, QC8) DWConv",4,0.039,10.130,94.026,0.0,19
6,"Copy (NC, X8)",4,0.016,4.156,98.182,0.0,29
7,"Max Pooling (NHWC, S8)",1,0.005,1.299,99.481,0.0,1
8,SOFTMAX,1,0.001,0.260,99.740,0.0,1
9,"Global Average Pooling (NWC, QS8)",1,0.001,0.260,100.000,0.0,1
10,RESHAPE,32,0.000,0.000,100.000,0.0,32


In [ ]:
filepath = models_dir.joinpath(f"{model_name}/{model_name}_layers.pkl")

#filepath = f"i:/tinyml/tiny_cnn/models/{model_name}/{model_name}_layers.pkl"


In [ ]:
try:
    df = pd.read_pickle(filepath)
except:
    print(f"Try reading the file as csv")
    df = pd.read_csv(filepath)

In [ ]:
df.columns

In [ ]:
df.head(10)

## Helper functions

In [ ]:
def split_tuples(x):
    x = str(x).strip("()")
    x = x.split(",")
    return x

In [ ]:
def return_batch_size(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)
    return split_strings[0].strip()

In [ ]:
def return_input_height(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [3,4]:
        return split_strings[1].strip()
    else:
        return "None"


In [ ]:
def return_input_width(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [4]:
        return split_strings[2].strip()
    else:
        return "None"


In [ ]:
def return_channels(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)

    return split_strings[-1].strip()

In [ ]:
def split_shape_columns(df, colum_name):
    df[colum_name] = df[colum_name].map(str)
    df[colum_name] = df[colum_name].str.strip("[]").replace('\)\,', ')*', regex=True)
    df_cols = df[colum_name].str.split("*", expand=True)
    prefix = colum_name.split("_")[0]
    df_cols.columns = [f"{prefix}_{x+1}" for x in df_cols.columns]
    return df_cols

In [ ]:
# df["input_shape"].unique()

In [ ]:
# split_df = df["input_shape"].map(str)
# split_df.unique

# Input shapes & activations

In [ ]:
df_input = split_shape_columns(df, "input_shape")
no_input_cols = len(df_input.columns)
for input in range(1,no_input_cols+1):
    df_input[f"b_i_{input}"]= df_input[f"input_{input}"].apply(return_batch_size)
    df_input[f"h_i_{input}"] = df_input[f"input_{input}"].apply(return_input_height)
    df_input[f"w_i_{input}"] = df_input[f"input_{input}"].apply(return_input_width)
    df_input[f"c_i_{input}"] = df_input[f"input_{input}"].apply(return_channels)


input_cols = df_input.columns.to_list()

#df_input["w_i_2"].replace('None', np.nan, inplace=True)
df_input["b_i_1"].replace('None', np.nan, inplace=True)
df_input["w_i_1"].replace('None', np.nan, inplace=True)
df_input["h_i_1"].replace('None', np.nan, inplace=True)
df_input["c_i_1"].replace('None', np.nan, inplace=True)

if "b_i_2" in input_cols:
    df_input["b_i_2"] = df_input["b_i_2"].str.replace("\(", "")
    df_input["b_i_2"].replace('None', np.nan, inplace=True)
    df_input["w_i_2"].replace('None', np.nan, inplace=True)
    df_input["h_i_2"].replace('None', np.nan, inplace=True)
    df_input["c_i_2"].replace('None', np.nan, inplace=True)
df_input.fillna(1, inplace=True)

# cast to int
df_input["w_i_1"] = df_input["w_i_1"].map(int)
df_input["h_i_1"] = df_input["h_i_1"].map(int)
df_input["b_i_1"] = df_input["b_i_1"].map(int)
df_input["c_i_1"] = df_input["c_i_1"].map(int)

if "b_i_2" in input_cols:
    df_input["b_i_2"] = df_input["b_i_2"].map(int)
    df_input["w_i_2"] = df_input["w_i_2"].map(int)
    df_input["h_i_2"] = df_input["h_i_2"].map(int)
    df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [ ]:
# df_input = split_shape_columns(df, "input_shape")
# df_input

In [ ]:
# #df["input_shape"][0] =(None, None, None, None)
# df["input_shape"] = df["input_shape"].map(str)
# df["input_shape"] = df["input_shape"].str.strip("[]").replace('\)\,', ')*', regex=True)
# df_input = df["input_shape"].str.split("*", expand=True)
# df_input.columns =[f"input_{x+1}" for x in df_input.columns]
# no_input_cols = len(df_input.columns)
# print(no_input_cols)
# df_input


In [ ]:
#df_input["input_1_split"] = df_input["input_1"].apply(split_tuples)

In [ ]:
# no_input_cols = len(df_input.columns)

In [ ]:
# for input in range(1,no_input_cols+1):
#     df_input[f"b_i_{input}"]= df_input[f"input_{input}"].apply(return_batch_size)
#     df_input[f"h_i_{input}"] = df_input[f"input_{input}"].apply(return_input_height)
#     df_input[f"w_i_{input}"] = df_input[f"input_{input}"].apply(return_input_width)
#     df_input[f"c_i_{input}"] = df_input[f"input_{input}"].apply(return_channels)

# df_input

In [ ]:
# input_cols = df_input.columns.to_list()


In [ ]:
# #df_input["w_i_2"].replace('None', np.nan, inplace=True)
# df_input["b_i_1"].replace('None', np.nan, inplace=True)
# df_input["w_i_1"].replace('None', np.nan, inplace=True)
# df_input["h_i_1"].replace('None', np.nan, inplace=True)
# df_input["c_i_1"].replace('None', np.nan, inplace=True)

# if "b_i_2" in input_cols:
#     df_input["b_i_2"] = df_input["b_i_2"].str.replace("\(", "")
#     df_input["b_i_2"].replace('None', np.nan, inplace=True)
#     df_input["w_i_2"].replace('None', np.nan, inplace=True)
#     df_input["h_i_2"].replace('None', np.nan, inplace=True)
#     df_input["c_i_2"].replace('None', np.nan, inplace=True)
# df_input.fillna(1, inplace=True)


In [ ]:
# df_input

In [ ]:
#df_input.c_i_1.value_counts()

In [ ]:

# df_input["w_i_1"] = df_input["w_i_1"].map(int)
# df_input["h_i_1"] = df_input["h_i_1"].map(int)
# df_input["b_i_1"] = df_input["b_i_1"].map(int)
# df_input["c_i_1"] = df_input["c_i_1"].map(int)

# if "b_i_2" in input_cols:
#     df_input["b_i_2"] = df_input["b_i_2"].map(int)
#     df_input["w_i_2"] = df_input["w_i_2"].map(int)
#     df_input["h_i_2"] = df_input["h_i_2"].map(int)
#     df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [ ]:

# df_input["b_i_1"] = df_input["b_i_1"].map(int)
# df_input["w_i_1"] = df_input["w_i_1"].map(int)
# df_input["h_i_1"] = df_input["h_i_1"].map(int)
# df_input["c_i_1"] = df_input["c_i_1"].map(int)

# df_input["b_i_2"] = df_input["b_i_2"].map(int)
# df_input["w_i_2"] = df_input["w_i_2"].map(int)
# df_input["h_i_2"] = df_input["h_i_2"].map(int)
# df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [ ]:
#df_input

# Output shape and activations

In [ ]:
df_output =split_shape_columns(df, "output_shape") # TODO: This split does not work for EfficientNet
no_output_cols = len(df_output.columns)
# split output dimensions
for output in range(1,no_output_cols+1):
    df_output[f"b_o_{output}"]= df_output[f"output_{output}"].apply(return_batch_size)
    df_output[f"h_o_{output}"] = df_output[f"output_{output}"].apply(return_input_height)
    df_output[f"w_o_{output}"] = df_output[f"output_{output}"].apply(return_input_width)
    df_output[f"c_o_{output}"] = df_output[f"output_{output}"].apply(return_channels)

# fill NaN with 1 for mulitplication
df_output["b_o_1"].replace('None', np.nan, inplace=True)
df_output["w_o_1"].replace('None', np.nan, inplace=True)
df_output["h_o_1"].replace('None', np.nan, inplace=True)
df_output["c_o_1"].replace('None', np.nan, inplace=True)
df_output.fillna(1, inplace=True)

df_output["w_o_1"] = df_output["w_o_1"].map(int)
df_output["h_o_1"] = df_output["h_o_1"].map(int)
df_output["b_o_1"] = df_output["b_o_1"].map(int)
df_output["c_o_1"] = df_output["c_o_1"].map(int)

In [ ]:
# df_output = split_shape_columns(df, "output_shape")
# df_output

In [ ]:
# no_output_cols = len(df_output.columns)
# no_output_cols

In [ ]:
# for output in range(1,no_output_cols+1):
#     df_output[f"b_o_{output}"]= df_output[f"output_{output}"].apply(return_batch_size)
#     df_output[f"h_o_{output}"] = df_output[f"output_{output}"].apply(return_input_height)
#     df_output[f"w_o_{output}"] = df_output[f"output_{output}"].apply(return_input_width)
#     df_output[f"c_o_{output}"] = df_output[f"output_{output}"].apply(return_channels)

# df_output

In [ ]:
# df_output["b_o_1"].replace('None', np.nan, inplace=True)
# df_output["w_o_1"].replace('None', np.nan, inplace=True)
# df_output["h_o_1"].replace('None', np.nan, inplace=True)
# df_output["c_o_1"].replace('None', np.nan, inplace=True)
# df_output.fillna(1, inplace=True)

In [ ]:
# df_output["w_o_1"] = df_output["w_o_1"].map(int)
# df_output["h_o_1"] = df_output["h_o_1"].map(int)
# df_output["b_o_1"] = df_output["b_o_1"].map(int)
# df_output["c_o_1"] = df_output["c_o_1"].map(int)

# Feature Engineering

In [ ]:
def get_unique_values_without_nan(df,col_name):
    """Retrieve list of unique column values, excluding nan

    Args:
        df (pd.DataFrame): pandas dataframe that contains the required column
        col_name (string): name of a column in the pandas dataframe

    Returns:
        list: list of extracted unique values of the specified column
    """
    try:
        value_list = df[col_name].unique().tolist()
        # exclude nan
        value_list = [item for item in value_list if str(item) != 'nan']
        return value_list
    except:
        print(f"WARNING: Colum name {col_name} is not in this dataframe!")

        return []

In [ ]:
df = df.join(df_input)

In [ ]:
df = df.join(df_output)

In [ ]:
name_code ="i_2"

In [ ]:
def calculate_activations(x, name_code):
    h_i = int(x[f"h_{name_code}"])
    w_i = int(x[f"w_{name_code}"])
    c_i = int(x[f"c_{name_code}"])
    out = h_i * w_i * c_i
    if out >1:
        return out
    else:
        return 0

In [ ]:
df["#i_1_activations"] = df.apply(calculate_activations, name_code= "i_1", axis=1)#df_input["h_i_1"] * df_input["w_i_1"] * df_input["c_i_1"]
df["#o_1_activations"] = df.apply(calculate_activations, name_code= "o_1", axis=1)#  df_output["h_o_1"] * df_output["w_o_1"] * df_output["c_o_1"]

In [ ]:
if "b_i_2" in input_cols:
    df["#i_2_activations"] = df.apply(calculate_activations, name_code="i_2", axis=1)

In [ ]:
df["#i_2_activations"].value_counts()

In [ ]:
if "#i_2_activations" in df.columns:
    df["peak_activations"] = df["#i_1_activations"] + df["#i_2_activations"] + df["#o_1_activations"]
else:
    df["peak_activations"] = df["#i_1_activations"]  + df["#o_1_activations"]    

In [ ]:
unique_values = get_unique_values_without_nan(df, "input_node_2")
unique_values

In [ ]:
# fill activations from start node to end node

for item in unique_values:
    first_idx = df.loc[df["name"] == item].index[0]
    last_idx = df.loc[df["input_node_2"] == item].index[0]

    activations = df["#i_2_activations"][last_idx]

    # fill the corresponding rows with the extracted information
    df.loc[first_idx: last_idx, "#i_2_activations"] = activations
print(df[first_idx: last_idx+1])

In [ ]:
df.columns

In [ ]:
df["total_params"] = df["params"].cumsum()
df["total_MACs"] = df["MACS"].cumsum()
df["total_OPS"] = df["OPS"].cumsum()

In [ ]:
# df["total_params"].max()
# df["peak_activations"].argmax()

In [ ]:
model_dict["model_name"] = model_name
model_dict["total_params"] = df["total_params"].max()
model_dict["total_MACs"] = df["total_MACs"].max()
model_dict["total OPS"] = df["total_OPS"].max()
model_dict["peak_activations"] = df["peak_activations"].max()
model_dict["index_peak_activations"] = df["peak_activations"].argmax()

In [ ]:
model_dict

In [ ]:
csv_file_name = "model_metrics.csv"
csv_path = Path.cwd().joinpath(csv_file_name)
append_dict_to_csv(csv_path, model_dict)

In [ ]:
# file path for the cleaned dataframe
df_filepath = models_dir.joinpath(f"{model_name}/{model_name}.pkl")

In [ ]:
df.to_pickle(df_filepath)

In [ ]:
df.columns

In [ ]:
df.head(12)

In [ ]:
df["input_shape"].unique()

# Working with Activations

In [ ]:
# unique_values = get_unique_values_without_nan(df, "input_node_0")
# unique_values

In [ ]:

# # fill activations from start node to end node

# for item in unique_values:
#     first_idx = df.loc[df["name"] == item].index[0]
#     last_idx = df.loc[df["input_node_0"] == item].index[0]

#     activations = df["#i_2_activations"][last_idx]

#     # fill the corresponding rows with the extracted information
#     df.loc[first_idx: last_idx, "#i_2_activations"] = activations
#     #print(df[first_idx: last_idx+1])

In [ ]:
# df.columns

In [ ]:
# activation_cols = ["index", "name", "input_shape", "input_node_1", "#i_1_activations" , "input_node_0", "#i_2_activations", "input_shape", "input_node_type_1", "input_node_type_0"]
# df_activations = df[activation_cols].copy()
# df_activations.head(20)

In [ ]:
# # fill activations from start node to end node

# for item in second_inputs:
#     first_idx = df_activations.loc[df_activations["name"] == item].index[0]
#     last_idx = df_activations.loc[df_activations["input_node_0"] == item].index[0]

#     activations = df_activations["#i_2_activations"][last_idx]
#     df_activations.loc[first_idx: last_idx, "#i_2_activations"] = activations
#     #print(df_activations[first_idx: last_idx+1])
    

    

In [ ]:
#input_name = "block_4_project_BN"

In [ ]:
# first_idx = df.loc[df["name"] == input_name].index[0]
# last_idx = df.loc[df["input_node_2"] == input_name].index[0]
# last_idx

In [ ]:
df[first_idx: last_idx+1]

In [ ]:
# df_activations.loc[last_idx]

In [ ]:
# activations = df_activations["#i_2_activations"][last_idx]
# activations

In [ ]:
# df_activations.loc[first_idx: last_idx, "#i_2_activations"] = activations
# df_activations.head(30)

# Save cleaned dataframe

In [ ]:
clean_df_filepath = models_dir.joinpath(f"{model_name}/{model_name}_layers_clean.pkl")

#filepath = f"i:/tinyml/tiny_cnn/models/{model_name}/{model_name}_layers.pkl"
df.to_pickle(clean_df_filepath)

# Load tflite model data

In [ ]:
# Get dataframes from benchmarking tflite model 
models_peak_memory_df_path = models_dir.joinpath(model_name, "peak_memory_df")
models_peak_memory_df_path

tensor_info_df_path = models_dir.joinpath(model_name, "tensor_info_df")
tensor_info_df_path

In [ ]:
tensor_info_df = pd.read_pickle(tensor_info_df_path)
tensor_info_df

In [ ]:
peak_memory_df = pd.read_pickle(models_peak_memory_df_path)
peak_memory_df

# Visualizations

In [ ]:
# fig = px.bar(df, x="name", y= ["#i_1_activations","#i_2_activations", "#o_1_activations"], text_auto=".2s",
# width=1400, height=500,
#     title=f"Peak activations per layer - {model_name}")
# fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#               annotation_position="bottom right")
# fig.update_layout(yaxis_range=[0,300000])
# #fig.update_layout(showlegend=True)
# fig.show()

In [ ]:
tflite_fig = px.bar(peak_memory_df, x="operator", y= ["peak_memory_kb"], text_auto=".2s",
width=1400, height=600,
    title=f"Peak activations per layer - tflite {model_name}")
tflite_fig.add_hline(y=256, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
              annotation_position="bottom right")
peak_memory = peak_memory_df["peak_memory_kb"].max()
tflite_fig.add_hline(y=peak_memory, line_width=3, line_dash="dash", line_color="red", annotation_text=f"{peak_memory} kB peak activation", 
              annotation_position="top")
tflite_fig.update_layout(yaxis_range=[0,300])
#tflite_fig.update_layout(showlegend=True)
tflite_fig.show()

In [ ]:
fig6 = px.bar(df, x="name", y= "MACS",
    title=f"MACs per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig6.show()

In [ ]:
fig2 = px.bar(df, x="name", y= "h_i_1", text_auto=".2s",
    title=f"Input height per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
#fig2.update_yaxes(autorange="reversed")
#fig2.update_yaxes(rangemode="tozero")
fig2.show()

In [ ]:
fig3 = px.bar(df, x="name", y= "c_i_1", text_auto=".2s",
    title=f"Channels per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
#fig3.update_yaxes(autorange="reversed")
#fig3.update_yaxes(rangemode="tozero")
# fig3.update_yaxes(
#     range=(0, 1280),
#     constrain='domain'
# )
fig3.show()

In [ ]:
fig4 = px.line(df, x="name", y= "total_params",
    title=f"Cumulative parameters per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig4.show()

In [ ]:
fig5 = px.line(df, x="name", y= "total_MACs",
    title=f"Cumulative MACs per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig5.show()

In [ ]:
# # Generate run ids
# id = wandb.util.generate_id()

# PROJECT = model_name.split("_")[0]

# run = wandb.init(
#         # Set the project where this run will be logged
#         project=PROJECT, 
#         name = model_name,
#         id = id, 
#         resume="allow",
#         sync_tensorboard=True
#         )
# # Specify the configuration variables
# config = wandb.config

# #config.batch_size = BATCH_SIZE
# #config.dropout =DROPOUT
# #config.learn_rate = LR
# #config.momentum = MOMENTUM
# #config.decay = 1e-6
# #config.epochs = EPOCHS
# #config.classes = classes
# config.id = id
# config.architecture = model_name

# # Create a table
# table = wandb.Table(columns = ["plotly_figure"])

# # Create path for Plotly figure
# path_to_plotly_html = "./plotly_figure.html"

# # Write Plotly figure to HTML
# fig.write_html(path_to_plotly_html, auto_play = False) # Setting auto_play to False prevents animated Plotly charts from playing in the table automatically

# # Add Plotly figure as HTML file into Table
# table.add_data(wandb.Html(path_to_plotly_html))

# # Log Table
# run.log({"Chart_table": table})

# #wandb.log({"Peak activations chart": fig})

# wandb.finish()